# Final Code

In [1]:
# --- Import Libraries ---
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --- Load Dataset ---
data = pd.read_csv("../datasets/cardekho_imputated.csv", index_col=[0])  

# Drop unnecessary columns
data.drop(['car_name', 'brand'], axis=1, inplace=True)

# Separate features and target
X = data.drop("selling_price", axis=1)
y = data["selling_price"]

# Apply LabelEncoder for 'model'
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

# Identify numeric and categorical columns
num_features = X.select_dtypes(exclude="object").columns
onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']

# Preprocessing: OneHotEncoding and Scaling
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, onehot_columns),
        ("StandardScaler", numeric_transformer, num_features)
    ], remainder='passthrough'
)

# --- Split Data into Training and Test Sets ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Apply preprocessing ---
# Fit the preprocessor on the training data only
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)  # Apply transform to X_test (no fitting)

# Save the preprocessor, model, and label encoder using joblib with compression
joblib.dump(preprocessor, '../backend/preprocessor.joblib', compress=9)  
joblib.dump(le, '../backend/label_encoder.joblib', compress=9) 

# --- Train the Random Forest model ---
model = RandomForestRegressor(
    n_estimators=500, 
    min_samples_split=5,
    min_samples_leaf=5,
    max_features=4, 
    max_depth=12, 
    n_jobs=-1
)

# model = RandomForestRegressor(
#     n_estimators=1000, 
#     min_samples_split=2, 
#     max_features=5, 
#     max_depth=15, 
#     n_jobs=-1
# )
# we can use above model with hyperparameters but after making joblib file it become 100+ mb size 
# and it give error of out of memory in render so we just do compress our model

model.fit(X_train_transformed, y_train)

# Save the trained model using joblib with compression
# joblib.dump(model, 'random_forest_model.joblib', compress=3)  # Save model with compression
joblib.dump(model, '../backend/random_forest_model.joblib', compress=9)  


# --- Evaluate Model ---
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

# Predictions
y_test_pred = model.predict(X_test_transformed)

# Evaluate model performance
mae, rmse, r2 = evaluate_model(y_test, y_test_pred)
print("\nModel Evaluation on Test Data:")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")



Model Evaluation on Test Data:
Mean Absolute Error (MAE): 110319.7753
Root Mean Squared Error (RMSE): 236526.5207
R² Score: 0.9257


In [2]:
# model = RandomForestRegressor(
#     n_estimators=1000, 
#     min_samples_split=2, 
#     max_features=5, 
#     max_depth=15, 
#     n_jobs=-1
# )

#  using this model we can get almost 94 r2 score